In [1]:
import requests
import os

In [2]:
def get_images(taxonKey, limit=10):
    # Definir a URL base da API do GBIF
    base_url = "https://api.gbif.org/v1/occurrence/search"
 
    # Substitua pelo UUID do publisher desejado
    publishing_org = "da86174a-a605-43a4-a5e8-53d484152cd3"
    
    # Parâmetros da requisição
    params = {
        "taxonKey": taxonKey,
        "mediaType": "StillImage",
        "publishingOrg": publishing_org,
        "limit": limit  # Limite de ocorrências por requisição
    }
    
    # Fazer a requisição GET
    response = requests.get(base_url, params=params)

    # Verificar se a requisição foi bem-sucedida
    if response.status_code == 200:
        data = response.json()
        occurrences = data["results"]
    else:
        print(f"Erro na requisição: {response.status_code}")
        occurrences = []
    
   # Filtrar URLs das imagens e identificar a parte da planta
    images_info = []
    for occurrence in occurrences:
        if "media" in occurrence:
            for media in occurrence["media"]:
                if media["type"] == "StillImage":
                    part = media.get("description", "Desconhecido")
                    images_info.append((media["identifier"], part))
    
    return images_info 

In [3]:
def download_images(images_info, base_download_path='images'):

    full_path = os.path.join('images', base_download_path)
    # Criar diretório base para baixar as imagens, se não existir
    if not os.path.exists(full_path):
        os.makedirs(full_path)
    
    # Baixar as imagens e organizar em pastas por 'part'
    for idx, (image_url, part) in enumerate(images_info):
        # Extrair a parte após o caractere ':'
        part_name = part.split(':')[-1].strip() if ':' in part else part
        
        # Criar o diretório para a parte específica, se não existir
        part_dir = os.path.join(full_path, part_name)
        if not os.path.exists(part_dir):
            os.makedirs(part_dir)
        
        # Baixar a imagem
        image_response = requests.get(image_url)
        if image_response.status_code == 200:
            image_path = os.path.join(part_dir, f'image_{idx+1}.jpg')
            with open(image_path, 'wb') as f:
                f.write(image_response.content)
        else:
            print(f"Erro ao baixar imagem {idx+1}: {image_response.status_code}")

In [4]:
images = get_images(2855860, 600)
download_images(images, 'cebolinha')